In [21]:
from ipyleaflet import Map, basemaps, Circle, Marker, Icon, MeasureControl, GeoData, AwesomeIcon, GeoJSON, AntPath, Polygon
from ipyleaflet.velocity import Velocity
import geopandas as gpd
from sidecar import Sidecar
import re
import math
import xarray as xr
import json
import threading
import queue
import time
import rtlsdr
import pyModeS
import random

from typing import List, Tuple, Dict


def calculate_destination_coordinates(lat, lon, cap, dist):
    # Earth radius in meters
    earth_radius = 6371000.0

    # Convert angles to radians
    lat_rad = math.radians(lat)
    lon_rad = math.radians(lon)
    cap_rad = math.radians(cap)

    # Calculate the destination latitude and longitude
    new_lat_rad = math.asin(math.sin(lat_rad) * math.cos(dist / earth_radius) +
                            math.cos(lat_rad) * math.sin(dist / earth_radius) * math.cos(cap_rad))

    new_lon_rad = lon_rad + math.atan2(math.sin(cap_rad) * math.sin(dist / earth_radius) * math.cos(lat_rad),
                                       math.cos(dist / earth_radius) - math.sin(lat_rad) * math.sin(new_lat_rad))

    # Convert radians back to degrees
    new_lat = math.degrees(new_lat_rad)
    new_lon = math.degrees(new_lon_rad)

    return new_lat, new_lon


def dms_to_decimal(coord_string):
    # Regular expression to extract values
    pattern = re.compile(r'(\d+)° (\d+)\' (\d+\.\d+)\" ([NSWE])')
    match = pattern.match(coord_string)

    if match:
        degrees, minutes, seconds, direction = match.groups()
        
        # Convert to decimal format
        decimal_coord = float(degrees) + float(minutes)/60 + float(seconds)/3600

        # Adjust for negative values (South or West)
        if direction in ['S', 'W']:
            decimal_coord = -decimal_coord

        return decimal_coord
    else:
        raise ValueError("Invalid coordinate string format")

def decimal_to_dms(latitude, longitude):
    # Function to convert decimal degrees to DMS format
    def dd_to_dms(degrees, is_latitude):
        direction = 'N' if is_latitude and degrees >= 0 else 'S'
        direction = 'E' if not is_latitude and degrees >= 0 else 'W'

        degrees = abs(degrees)
        minutes, seconds = divmod(degrees * 3600, 60)
        degrees, minutes = divmod(minutes, 60)

        return f"{int(degrees)}° {int(minutes)}' {seconds:.2f}\" {direction}"


In [ ]:
from dataclasses import dataclass

# Class to store XWing Plane's current state
@dataclass
class XWingPlaneState:
    latitude: float
    longitude: float
    altitude: float
    roll: float
    pitch: float
    yaw: float
    route: float
    heading: float
    speed_north: float
    speed_east: float
    speed_vertical: float

# Class to store data for "danger zones"
class DangerZone:
    # Delay before activating termination system (in seconds)
    _delay_to_termination: float
    # Input state used to calculate the danger zone
    _xwing_plane_state: XWingPlaneState
    # Coordinates defining the contour of the zone (100 points)
    _contour_points: List[Tuple[float, float]]

def __init__(self,
             p_delay_to_termination: float,
             p_xwing_plane_state: XWingPlaneState):
    self._delay_to_termination = p_delay_to_termination
    self._xwing_plane_state = p_xwing_plane_state
    self._contour_points = []
    self.compute_contour()
    
def compute_contour(self):
    return [(self._xwing_plane_state.latitude - 0.1 * (1 + self._delay_to_termination), self._xwing_plane_state.longitude - 0.1 * (1 + self._delay_to_termination),
            (self._xwing_plane_state.latitude + 0.1 * (1 + self._delay_to_termination, self._xwing_plane_state.longitude - 0.1 * (1 + self._delay_to_termination),
            (self._xwing_plane_state.latitude + 0.1 * (1 + self._delay_to_termination, self._xwing_plane_state.longitude + 0.1 * (1 + self._delay_to_termination),
            (self._xwing_plane_state.latitude - 0.1 * (1 + self._delay_to_termination, self._xwing_plane_state.longitude + 0. * (1 + self._delay_to_termination1)]
    

In [ ]:
# Class to simulate the processing and sending of data to update the Map and its elements
class SimulatedSendingThread(thread.Threading):
    # Queue
    _queue: queue.Queue
    # Count of many iterations have been done (used to simulatedifferent phases of a flight)
    _counter: int
    # Interval between sending 2 messages
    _interval: float
    # XWing Plane State - previous
    _xwing_previous_state: XWingPlaneState
    # XWing Plane State - updated
    _xwing_updated_state: XWingPlaneState
    # Danger Zones:
    _danger_zones: Dict[float, DangerZone]

    def __init__(self,
                 p_queue: queue.Queue,
                 p_interval: float):
        super().__init__()
        self._queue = p_queue
        self._interval = p_interval
        dms_lat_cazaux = "44° 31' 41.33\" N"
        dms_lon_cazaux = "1° 08' 15.95\" W"
        self._danger_zones = {}
        self._xwing_previous_state = XWingPlaneState(latitude=dms_to_decimal(dms_lat_cazaux),
                                                     longitude=dms_to_decimal(dms_lon_cazaux))
        self._xwing_updated_state = self._xwing_previous_state
        self.daemon = True  # Set as daemon to exit with the main program

    def run(self):
        while True:
            # Simulate some background work and update the queue
            message = self.compute_message()
            self.message_queue.put(message)
            self._counter += 1
            # Sleep for a while (simulating a delay between updates)
            time.sleep(self._interval + random.uniform(0, 0.5 * self._interval))

    def compute_message(self):
        self._xwing_updated_state = self.simulate_update_plane_state()
        self._danger_zones = self.compute_danger_zones()
        return {'state': self._xwing_updated_state, 'danger_zones': self._danger_zones}

    def simulate_update_plane_state():
        previous_state = self._xwing_previous_state
        previous_state.latitude += 0.02
        previous_state.longitude += 0.015
        return previous_state

    def compute_danger_zones(self):
        for termination_delay in [0, 5, 15]:
            danger_zone = DangerZone(termination_delay, self._xwing_updated_state)
            self._danger_zones[termination_delay] = danger_zone
            
 

In [30]:
# Coordinates or roughly the starting tip of the landing strip in Cazaux
dms_lat_cazaux = "44° 31' 41.33\" N"
dms_lon_cazaux = "1° 08' 15.95\" W"

center = [dms_to_decimal(dms_lat_cazaux), dms_to_decimal(dms_lon_cazaux)]

# Icon and Marker for the XWING Plane
xwing_plane_icon = Icon(
    icon_url='http://localhost:8888/files/xwing_red_xs.svg?_xsrf=2%7Caf9410d2%7C287e68d8c54a4f39fe32864d93e70913%7C1706357870',
    icon_size=[40, 40],
    icon_anchor=[20, 20]
)
xwing_plane_marker = Marker(location=center,
                             draggable=False,
                             icon=xwing_plane_icon,
                             rotation_angle=56.4,
                             rotation_origin='20px 20px')

# Measure Control (measure distances/areas). Has issues though
measure = MeasureControl(
    position='bottomleft',
    active_color = 'orange',
    primary_length_unit = 'kilometers'
)
measure.add_area_unit('sqkmeters', 1000000, 4)
measure.primary_area_unit='sqkmeters'

# Just to show we can add Layer for wind
ds = xr.open_dataset("wind-global.nc")
display_options = {
    "velocityType": "Global Wind",
    "displayPosition": "bottomright",
    "displayEmptyString": "No wind data",
}
wind = Velocity(
    data=ds,
    zonal_speed="u_wind",
    meridional_speed="v_wind",
    latitude_dimension="lat",
    longitude_dimension="lon",
    velocity_scale=0.01,
    max_velocity=20,
    display_options=display_options,
)

# Dsplay habiatetd zones from a GeoJSON file 
with open('habitations.geojson','r') as f:
    data = json.load(f)
geo_json = GeoJSON(data=data,
                   style = {'color': 'Red',
                            'opacity':1,
                            'weight':1.9,
                            'dashArray':'9',
                            'fillOpacity':0.3})

m = Map(basemap=basemaps.OpenStreetMap.Mapnik,
        center=center,
        zoom=12,
        scroll_wheel_zoom=True,
       interpolation='nearest')

# m.add(wind)
m.add(measure)
m.add(xwing_plane_marker)
m.add(geo_json)

measure.completed_color = 'red'

sc = Sidecar(title="XWINg Map")

with sc:
    display(m)